### only install once

In [ ]:
%%bash
## only install once

#pip3 install awscli --upgrade --user
sudo apt-get install awscli -y
pip3 install awscli boto3 --upgrade --user



### please fill out the following AWS credential so we can get the ECR docker container

In [ ]:
import os

os.environ['AWS_ACCESS_KEY_ID']=""
os.environ['AWS_SECRET_ACCESS_KEY']=""
os.environ['AWS_SESSION_TOKEN']=""
# Set your ECR details as environment variables
os.environ['ECR_ACCOUNT_ID']=''  # Replace with your actual account ID
os.environ['ECR_REGION']='us-east-1'           # Replace with your actual region
os.environ['ECR_IMAGE_NAME']='ros-humble-greengrass-minipupper'  # Replace with your image name
os.environ['ECR_TAG']='latest'                       # Replace with your image tag


from string import Template
import os
from pathlib import Path

template_string = """
[default]
aws_access_key_id=${AWS_ACCESS_KEY_ID}
aws_secret_access_key=${AWS_SECRET_ACCESS_KEY}
aws_session_token=${AWS_SESSION_TOKEN}

"""

template = Template(template_string)
filled_content = template.substitute(os.environ)

# Define the AWS credentials file path
aws_credentials_path = Path.home() / '.aws' / 'credentials'

# Ensure the .aws directory exists
aws_credentials_path.parent.mkdir(parents=True, exist_ok=True)

# Write the content to the AWS credentials file
with open(aws_credentials_path, "w") as f:
    f.write(filled_content)


In [ ]:
def retag_docker_image(old_image_tag, new_image_tag):
    try:
        # Run the docker tag command
        subprocess.run(['docker', 'tag', old_image_tag, new_image_tag], check=True)
        print(f"Successfully retagged {old_image_tag} to {new_image_tag}")
    except subprocess.CalledProcessError as e:
        print(f"Error retagging image: {e}")

In [ ]:
import base64
# Initialize a session using boto3
session = boto3.Session()
ecr_client = session.client('ecr')

# Get the authorization token
token_response = ecr_client.get_authorization_token()
auth_data = token_response['authorizationData'][0]
registry = auth_data['proxyEndpoint']
authorization_token = auth_data['authorizationToken']

# Decode the token (it's base64 encoded)
decoded_token = base64.b64decode(authorization_token).decode('utf-8')
username, password = decoded_token.split(':')

# Log in to ECR using Docker with --password-stdin
try:
    process = subprocess.Popen(
        ['docker', 'login', '--username', username, '--password-stdin', registry],
        stdin=subprocess.PIPE,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
    )
    # Send the password to the subprocess
    stdout, stderr = process.communicate(input=password.encode())

    if process.returncode == 0:
        print("Login successful!")
    else:
        print(f"Error during Docker login: {stderr.decode().strip()}")
except Exception as e:
    print(f"An error occurred: {e}")

# Pull the Docker image using environment variables
image_name = f"{os.getenv('ECR_ACCOUNT_ID')}.dkr.ecr.{os.getenv('ECR_REGION')}.amazonaws.com/{os.getenv('ECR_IMAGE_NAME')}:{os.getenv('ECR_TAG')}"
subprocess.run(['docker', 'pull', image_name])
local_image = f"{os.getenv('ECR_IMAGE_NAME')}:{os.getenv('ECR_TAG')}"
retag_docker_image(image_name, local_image)

In [ ]:
!docker-compose up

In [ ]:
!docker-compose down